# Environment

### Import required packages

In [ ]:
from apiclient.discovery import build
import dotenv
from googleapiclient.http import MediaIoBaseDownload
from httplib2 import Http
import io
import json
from oauth2client.service_account import ServiceAccountCredentials
import os

### Load .env variables inside custom file

In [ ]:
PROJECT_PATH = os.path.dirname(os.getcwd())
dotenv.load_dotenv(
    dotenv_path=os.path.join(PROJECT_PATH, '.env'),
    verbose=True
)

### Declare env vars globally

In [ ]:
project_id = os.getenv("GOOGLE_PROJECT")
client_id = os.getenv("GOOGLE_CLIENT_ID")
private_key_id = os.getenv("GOOGLE_PRIVATE_KEY_ID")
private_key = os.getenv("GOOGLE_PRIVATE_KEY")
main_folder_id = os.getenv("MAIN_FOLDER_ID")

# Google Drive API

### Create json file for auth

In [ ]:
def create_dict_auth(
    project_id_: str = project_id,
    client_id_: str = client_id,
    private_key_id_: str = private_key_id,
    private_key_: str = private_key
) -> None:
    """
    Create the json file to authenticate
    python to acess Google Sheet API.
    """
    key_json = {}
    # Decode key to remove '//'
    decode_key = bytes(private_key_, "utf-8").decode("unicode_escape")
    # Some default variables
    api = "googleapis.com"
    py_con = "dark-service"
    # Declaring each key-value pair
    key_json["type"] = "service_account"
    key_json["project_id"] = project_id_
    key_json["private_key_id"] = private_key_id_
    key_json["private_key"] = decode_key
    key_json["client_email"] = f"{py_con}@{project_id_}.iam.gserviceaccount.com"
    key_json["client_id"] = client_id_
    key_json["auth_uri"] = "https://accounts.google.com/o/oauth2/auth"
    key_json["token_uri"] = f"https://oauth2.{api}/token"
    key_json["auth_provider_x509_cert_url"] = f"https://www.{api}/oauth2/v1/certs"
    key_json["client_x509_cert_url"] = f"https://www.{api}/robot/v1/metadata/x509/{py_con}%40{project_id}.iam.gserviceaccount.com"
    # Save dict as a json file
    with open(os.path.join(PROJECT_PATH, 'google_service_account.json'), 'w') as fp:
        json.dump(key_json, fp)

In [ ]:
# Creating json through custom function
json_auth = create_dict_auth()

### Set up the credentials

1 - Activate Google Drive API \
2 - Create service account \
3 - Grant access to Google Cloud project with your personal email \
4 - Add service account key \
3 - Share google drive with client_email

##### Create client config

In [ ]:
# Authenticate with json keyfile
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    os.path.join(PROJECT_PATH, 'google_service_account.json'),
    scopes=['https://www.googleapis.com/auth/drive']
)
# Authorize with http connection
http_auth = credentials.authorize(Http())
drive = build('drive', 'v3', http=http_auth)

##### Check connection with ai-shorts folder in Google Drive

In [ ]:
# From project folder, print files/folders
request = drive.files().list(
    orderBy="modifiedTime desc",
    q=f"'{main_folder_id}' in parents",
    fields="files(id, name)"
).execute()
subfolders = request.get('files', [])
for f in subfolders:
    print(f)

# Store subfolder id
target_subfolder = subfolders[0]["id"]
target_subfolder

In [ ]:
# Extract video id from subfolder id
request_2 = drive.files().list(
    pageSize=5,
    orderBy="modifiedTime desc",
    q=f"'{target_subfolder}' in parents",
    fields="files(id, name)"
).execute()
subfolders_2 = request_2.get('files', [])
for sub in subfolders_2:
    print(sub)

# Get result
file_id = next((dict_['id'] for dict_ in subfolders_2 if dict_['name'].endswith('.mp4')), None)
file_id

##### Extracting video and upload to github workspace

In [ ]:
# Request content
request = drive.files().get_media(fileId=file_id)
file_name = drive.files().get(fileId=file_id).execute()['name']

# Define path to save
file_path = os.path.join(PROJECT_PATH, file_name)
fh = io.FileIO(file_path, 'wb')

# Download content
downloader = MediaIoBaseDownload(fh, request)
done = False
while not done:
    status, done = downloader.next_chunk()

print(f"Download progress: {status.progress() * 100:.2f}%")
print(f"File downloaded to: {file_path}")